In [1]:
import os
import sys
import json
import tqdm
from dotenv import load_dotenv
load_dotenv()

project_root = os.path.abspath(os.path.join(os.path.dirname('../')))
sys.path.append(project_root)

import core.data_schema.model as model
from core.database import Database
db = Database()

In [ ]:
import pprint as pp
gallery = model.Gallery.objects().first()
# output_data = pop_gallery.to_json()
print(gallery.gid)
for m in gallery.models:
  print(f'name: {m.name}')
  print('tags: ')
  for t in m.tags:
    print(t.name)



In [3]:
galleries = model.Gallery.objects(__raw__={"$where": "this.channels.length >= 2"})


In [4]:
model_name='Ashley Adams'
model_id = "647c36ffa6f57b54a3e18159"
# galleries = model.Gallery.objects(channels__size=2)
for gallery in tqdm.tqdm(galleries, desc="Update Channel's parent index"):
  channels = gallery.channels
  has_new_child = False
  for i in range(1, len(channels)):
    if channels[i] not in channels[0].children:
      has_new_child = True
      channels[0].children.append(channels[i])
    if channels[i].parent != channels[0]:
      channels[i].parent = channels[0]
      channels[i].save()
      print('save new parent')
  if has_new_child: 
    print('has new child')
    channels[0].save()
# for g in galleries:
#   print(g.id)
#   print(g.path)  
#   print(g.gid)
#   print(g.models)
#   print("------------------")
  
# model.Model.objects(id=model_id)

Update Channel's parent index:   1%|▏         | 5404/422907 [00:34<26:32, 262.21it/s]  